In [ ]:
import os
import json
import random
import torchaudio
from tqdm import tqdm
from glob import glob
from multiprocessing import Pool
files = glob("/scratch4/users/od/YTCharts/audios/*.wav")
random.seed(1019)

indices = [i for i in range(len(files))]
random.shuffle(indices)


# train_indices = indices[:int(0.8 * len(indices))]
# valid_indices = indices[int(0.8 * len(indices)):int(0.9 * len(indices))]
# test_indices = indices[int(0.9 * len(indices)):]
train_indices = indices[:int(0.9 * len(indices))]
valid_indices = []
test_indices = indices[int(0.9 * len(indices)):]


train_indices = train_indices
valid_indices = valid_indices
test_indices = test_indices

TARGET_SR = 44100


def runner(wav_path):
    name = os.path.basename(wav_path)[:-4]
    no_vocal_path = os.path.join(f"/scratch4/users/od/YTCharts/separated/htdemucs/{name}/no_vocals.mp3")
    link_path = os.path.join(f"/scratch4/users/od/YTCharts/v1/{split_name}/{name}.mp3")
    json_path = link_path.replace(".mp3", ".json")

    if (os.path.exists(link_path) and os.path.exists(json_path)):
        return

    if (not os.path.exists(no_vocal_path)):
        print("missing origin audio file", no_vocal_path)
        return

    os.system(f"ln -sf {no_vocal_path} {link_path}")

    wav, sr = torchaudio.load(link_path)

    assert sr == TARGET_SR, "unexpected sample rate."

    template = {
        "key": "",
        "artist": "",
        "sample_rate": sr,
        "file_extension": "mp3",
        "description": "",
        "keywords": [],
        "duration": wav.shape[-1] / sr,
        "bpm": "",
        "genre": "",
        "title": "",
        "name": name,
        "instrument": "Mix",
        "moods": []
    }

    with open(json_path, "w") as f:
        json.dump(template, f)


for split_name, split_indices in dict(train=train_indices, valid=valid_indices, test=test_indices).items():
    with Pool(16) as p:
        for _ in tqdm(
            p.imap_unordered(
                runner,
                [files[idx] for idx in split_indices]
            ),
            total=len(split_indices)
        ):
            continue

In [6]:
import os
from glob import glob
files = glob(f"/scratch4/users/od/YTCharts/v1/*/*.mp3")
file = files[:1]
for file in files:
    video_dst = file.replace(".mp3", ".mp4")
    video_src = file.replace(".mp3", ".mp4").replace("v1/train", "videos").replace("v1/test", "videos")
    name = os.path.basename(file)[:-4]
    os.system(f"ln -s {video_src} {video_dst}")

In [5]:
import os
from glob import glob
files = glob(f"/scratch4/users/od/YTCharts/v1/*/*.mp4")
# files = files[:1]
for file in files:
    os.system(f'rm "{file}"')